Most of this code is from the official YOLOv8 [tutorial](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb#scrollTo=tdSMcABDNKW-).

## Install YOLOv8

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from IPython.display import display, Image
from google.colab.patches import cv2_imshow

model = YOLO('yolov8m.pt')

## Extract frames for all videos

In [ ]:
os.mkdir("/content/frames/")

In [ ]:
import shutil
import subprocess
from tqdm import tqdm
path = "/content/drive/MyDrive/ataxia_dataset/raw_videos_ver2/"
# Extract frames for all videos & save them
for video in os.listdir(path):
  if ".mov" not in video:
    continue
  else:
    # Break the video into frames at 30 fps
    if os.path.exists(f"/content/frames/{video[:-4]}/"):
      continue
    os.mkdir(f"/content/frames/{video[:-4]}/")
    command = ["ffmpeg", "-i", os.path.join(path, video), "-vf", "fps=30", f"/content/frames/{video[:-4]}/output_%05d.jpg"]
    out = subprocess.run(command, stderr=subprocess.PIPE)

## Iterate over videos sequentially, get bounding boxes & resize

In [ ]:
os.mkdir("/content/updated_frames/")
os.mkdir("/content/updated_videos/")

In [ ]:
SP_LIST = [28, 29, 31, 32, 33, 36, 38, 41, 45, 47, 48, 50, 51, 52, 53, 54, 55, 141]

In [ ]:
NORMALIZED_HEIGHT = 800
cur_video_num = 2

for cur_video_num in tqdm(range(0, 151)):
  if cur_video_num in SP_LIST:
    make_vid = ["ffmpeg", "-framerate", "30", "-pattern_type", "glob", "-i", f'/content/frames/{cur_video_num}/*.jpg', \
                "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2", "-c:v", "libx264", "-pix_fmt", "yuv420p", \
                f"/content/updated_videos/out_{cur_video_num}.mp4"]
    out = subprocess.run(make_vid, stderr=subprocess.PIPE)
    continue
  if not os.path.exists(f"/content/frames/{cur_video_num}/"):
    continue
  for frame in os.listdir(f"/content/frames/{cur_video_num}/"):
    if os.path.exists(f"/content/updated_frames/{cur_video_num}/updated_{frame}"):
      continue
    # Define input image
    image = cv2.imread(f"/content/frames/{cur_video_num}/{frame}")

    # Get image dimensions
    (height, width) = image.shape[:2]

    # Get bounding boxes around objects
    results = model.predict(source=image, iou=0.7, verbose=False)

    # Initialize the final placeholders
    max_conf = -1
    max_conf_person = None

    # Loop over the detections
    for result in results:
      for (xywh, confidence, cls) in zip(result.boxes.xywh, result.boxes.conf, result.boxes.cls):

        if cls == 0 and confidence > max_conf:
            # Object detected
            center_x = int(xywh[0]) # * width
            center_y = int(xywh[1]) # * height
            w = int(xywh[2]) # * width
            h = int(xywh[3]) # * height

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Add the detection to the list of people
            max_conf_person = (x, y, w, h)
            max_conf = confidence

    if max_conf_person == None:
      print(f"Error at frame : {frame} in video : {cur_video_num}.")
      continue
    x, y, w, h = max_conf_person
    cropped_image = image[y:y+h, x:x+w]
    aspect_ratio = w/h
    new_width = int(NORMALIZED_HEIGHT * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(cropped_image, (new_width, NORMALIZED_HEIGHT))

    if not os.path.exists(f"/content/updated_frames/{cur_video_num}/"):
      os.mkdir(f"/content/updated_frames/{cur_video_num}/")
    cv2.imwrite(f"/content/updated_frames/{cur_video_num}/updated_{frame}", resized_image)
  make_vid = ["ffmpeg", "-framerate", "30", "-pattern_type", "glob", "-i", f'/content/updated_frames/{cur_video_num}/*.jpg', \
              "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2", "-c:v", "libx264", "-pix_fmt", "yuv420p", \
              f"/content/updated_videos/out_{cur_video_num}.mp4"]
  out = subprocess.run(make_vid, stderr=subprocess.PIPE)

## Save `updated_videos/`
Zipping just to be safe.

In [ ]:
!zip -r updated_videos.zip updated_videos/

In [ ]:
shutil.copy(f"/content/updated_videos/", "/content/drive/MyDrive/ataxia_dataset/")

# Thanks!